<a href="https://colab.research.google.com/github/bothananth-vb/LIME-ML/blob/main/IEEE_TASK_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv -O diabetes.csv

--2024-10-16 16:28:16--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘diabetes.csv’

diabetes.csv        100%[===================>]  22.73K  --.-KB/s    in 0s      

2024-10-16 16:28:17 (57.6 MB/s) - ‘diabetes.csv’ saved [23278/23278]



In [2]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=7b127e55824844206021e7765c55f63fbe52d74c2026b8720f6ec2a07a8d0f61
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


data.columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]  # Replace with your actual column names


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lime.lime_tabular import LimeTabularExplainer

# Load the dataset
data = pd.read_csv('diabetes.csv')
data.columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
                "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]

# Split the data into features and target
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nModel Evaluation:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

# Create a LIME explainer
explainer = LimeTabularExplainer(X_train.values,
                                 feature_names=X_train.columns,
                                 class_names=['No Diabetes', 'Diabetes'],
                                 discretize_continuous=True)

# Explain multiple instances and print the explanations in readable format
num_instances_to_explain = 5
for i in range(num_instances_to_explain):
    instance_to_explain = X_test.iloc[i:i+1]  # Keep as a DataFrame with feature names
    explanation = explainer.explain_instance(instance_to_explain.values[0], model.predict_proba)

    print(f"\nExplanation for Instance {i+1} (Actual class: {y_test.iloc[i]}):")
    explanation_dict = dict(explanation.as_list())

    print(f"{'Feature':<30} {'Contribution to Prediction':>30}")
    print("-" * 60)
    for feature, weight in explanation_dict.items():
        print(f"{feature:<30} {weight:>30.4f}")

    # Fix the warning by reshaping the instance into a DataFrame before passing it to predict_proba
    prediction_proba = model.predict_proba(instance_to_explain)[0][1]  # Still use a DataFrame for prediction

    print(f"\nPrediction probability for Diabetes: {prediction_proba:.4f}")
    print("-" * 60)



Model Evaluation:
Accuracy:  0.7857
Precision: 0.7143
Recall:    0.7018
F1-score:  0.7080

Explanation for Instance 1 (Actual class: 0):
Feature                            Contribution to Prediction
------------------------------------------------------------
Glucose <= 99.00                                      -0.2034
Age > 41.00                                            0.0707
32.00 < BMI <= 36.60                                   0.0531
Insulin > 127.00                                       0.0422
SkinThickness > 32.00                                  0.0312
3.00 < Pregnancies <= 6.00                             0.0047
BloodPressure <= 62.00                                -0.0041
0.38 < DiabetesPedigreeFunction <= 0.63                         0.0020

Prediction probability for Diabetes: 0.3600
------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



Explanation for Instance 2 (Actual class: 0):
Feature                            Contribution to Prediction
------------------------------------------------------------
Glucose > 139.00                                       0.2881
BMI <= 27.40                                          -0.1773
Age <= 24.00                                          -0.1013
DiabetesPedigreeFunction <= 0.24                        -0.0576
Pregnancies <= 1.00                                   -0.0465
Insulin > 127.00                                       0.0382
62.00 < BloodPressure <= 72.00                        -0.0213
0.00 < SkinThickness <= 23.00                         -0.0146

Prediction probability for Diabetes: 0.2900
------------------------------------------------------------

Explanation for Instance 3 (Actual class: 0):
Feature                            Contribution to Prediction
------------------------------------------------------------
99.00 < Glucose <= 117.00                             -0

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



Prediction probability for Diabetes: 0.1500
------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



Explanation for Instance 4 (Actual class: 0):
Feature                            Contribution to Prediction
------------------------------------------------------------
BMI <= 27.40                                          -0.1952
99.00 < Glucose <= 117.00                             -0.0949
Pregnancies > 6.00                                     0.0914
DiabetesPedigreeFunction > 0.63                         0.0860
29.00 < Age <= 41.00                                   0.0528
SkinThickness <= 0.00                                  0.0082
Insulin <= 0.00                                       -0.0047
72.00 < BloodPressure <= 80.00                        -0.0027

Prediction probability for Diabetes: 0.1500
------------------------------------------------------------

Explanation for Instance 5 (Actual class: 0):
Feature                            Contribution to Prediction
------------------------------------------------------------
BMI <= 27.40                                          -0.